# Notebook 07: Multi-Agent Basics

## Learning Objectives
- Import and use agents from src/agents/
- Build a 2-agent pipeline (Solver + Critic)
- Implement message passing and logging
- Analyze complete trace output

## Multi-Agent System Overview

```
Problem
  └── Solver Agent
       └── Step-by-step solution
            └── Critic Agent
                 └── VERDICT: CORRECT / INCORRECT
                      └── (If INCORRECT) Reviser Agent
                           └── Corrected solution
                                └── Verifier Agent
                                     └── ANSWER: [NUMBER]
```

In [ ]:
# !pip install torch transformers

In [ ]:
import sys
sys.path.insert(0, '..')
from src.agents import SolverAgent, CriticAgent, ReviserAgent, VerifierAgent
from src.orchestration.logger import TraceLogger
import json
print('All agents imported!')

## Step 1: Create and Test Individual Agents

In [ ]:
solver   = SolverAgent(agent_id='solver_0')
critic   = CriticAgent(agent_id='critic_0')
reviser  = ReviserAgent(agent_id='reviser_0')
verifier = VerifierAgent(agent_id='verifier_0')

print('Agents created:')
for agent in [solver, critic, reviser, verifier]:
    print(f'  {agent}')

In [ ]:
problem = 'A store has 45 apples. It sells 18. How many remain?'
print(f'Problem: {problem}\n')

# Solver
solver_result = solver.call(problem)
print('Solver output:')
print(solver_result['response'])
print(f'Latency: {solver_result["latency_ms"]:.1f} ms')

In [ ]:
# Critic
critic_result = critic.call(solver_result['response'])
print('Critic output:')
print(critic_result['response'])
print('Verdict:', critic.extract_verdict(critic_result['response']))

In [ ]:
# Verifier
veri = verifier.verify(solver_result['response'], ground_truth=27.0)
print('Verifier:')
print(f'  Extracted answer: {veri["extracted_answer"]}')
print(f'  Correct: {veri["correct"]}')

## Step 2: Run Full Pipeline with Logging

In [ ]:
from src.orchestration.pipeline import PipelineOrchestrator

# Reset agents
for a in [solver, critic, reviser, verifier]: a.reset()

pipeline = PipelineOrchestrator(
    agents=[solver, critic, reviser, verifier],
    max_rounds=2,
)
result = pipeline.run(problem, ground_truth=27.0)

print('Pipeline Result:')
print(json.dumps({k: v for k, v in result.items() if k != 'trace_summary'}, indent=2, default=str))
print('\nTrace summary:')
print(json.dumps(result['trace_summary'], indent=2))

---

## Exercises

1. **Add agents:** Add a second solver and compare their solutions
2. **Different problem:** Test on all 5 GSM8K sample problems. What accuracy do you get?
3. **Trace analysis:** Which agent has the most messages? Highest latency?
4. **Round limit:** Set max_rounds=1. How does accuracy change?
5. **Extension:** Add a timeout mechanism (skip an agent if it takes too long)